In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

torch_device = "cuda" if torch.cuda.is_available() else "cpu"
path = "C:\\Users\\deep\\Desktop\\demo2025\\output"
#path = "C:\\Users\\deep\\Desktop\\demo2025\\gpt2"
tokenizer = AutoTokenizer.from_pretrained(path)
# to inference from default GPT2 model:
model = AutoModelForCausalLM.from_pretrained(path, pad_token_id=tokenizer.eos_token_id).to(torch_device)

# to inference from fine-tuned GPT2 model:
#model = AutoModelForCausalLM.from_pretrained(path, pad_token_id=tokenizer.eos_token_id).to(torch_device)

# what sample text to generate from
model_inputs = tokenizer('I need to cancel my order please', return_tensors='pt').to(torch_device)

# generate 100 tokens
greedy_output = model.generate(**model_inputs, max_new_tokens=100)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I need to cancel my order please.
Assistant: I'm sorry to hear that. I'm here to help.
Assistant: I'm here to help you with any problems you may have.
User: I'm sorry to hear that. I'm here to help you with any problems you may have.
Assistant: I'm here to help you with any problems you may have.
Assistant: I'm here to help you with any problems you may have.
Assistant: I'm here to help you with any problems


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments


def fine_tune_gpt2(model_name, train_file, output_dir):
    # Load GPT-2 model and tokenizer
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)

    # Load training dataset
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_file,
        block_size=128)
    # Create data collator for language modeling
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False)
    # Set training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=5,
        per_device_train_batch_size=4,
        save_steps=10_000,
        save_total_limit=2,
    )
    # Train the model
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )
    trainer.train()
    # Save the fine-tuned model
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)


# Start fine-tune and save the model to the output directory
fine_tune_gpt2("gpt2", "./mental_health_data.txt", r"C:\Users\deep\Desktop\demo2025\output")